# **Exercice 1 : Utilisation de load_summarize_chain**
Dans cet exercice, vous allez utiliser la fonction load_summarize_chain pour résumer un document PDF. Cette approche est très utile lorsque vous travaillez avec des documents volumineux que vous souhaitez résumer automatiquement.

**Instructions :**
1. Charger et découper un document PDF : Vous allez utiliser un PDF, le découper en parties plus petites pour le rendre plus facile à traiter.
2. Configurer un modèle de langage (LLM) : Initialisez un modèle basé sur GPT pour générer le résumé.
3. Utiliser load_summarize_chain : Mettez en place une chaîne de résumé avec l'approche de raffinement (refine).
4. Résumer les documents découpés : Utilisez cette chaîne pour résumer les différents morceaux du document et afficher le résumé final.

In [24]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from dotenv import load_dotenv

In [25]:
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')
langchain_key = os.getenv('LANGCHAIN_API_KEY')

In [26]:
# Chargement du PDF
loader = PyPDFLoader("LM_Solene_Degrutere.pdf")
pages = loader.load()

# Découpage en chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(pages)


In [27]:
llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')

In [28]:
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template="""
Tu es un expert du résumé. Améliore le résumé existant ci-dessous en y ajoutant des informations pertinentes tirées du nouveau texte fourni.
Résumé existant : {existing_answer}
Nouveau texte : {text}
Résumé amélioré :"""
)

initial_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
Résume le texte suivant en français de manière concise et claire :
Texte : {text}
Résumé :"""
)

In [29]:
chain = load_summarize_chain(
    llm,
    chain_type="refine",
    question_prompt=initial_prompt,
    refine_prompt=refine_prompt,
    verbose=True
)

In [30]:
result = chain.run(docs)
print(result)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Résume le texte suivant en français de manière concise et claire :
Texte : Degrutère Solène 
93200 Saint Denis  
solene.degrutere@orange.fr  
06.48.44.42.72  
Objet : Candidature spontanée pour un poste dans le domaine du Traitement Automatique du Langage 
 Madame, Monsieur, 
Par la présente, je me permets de vous adresser ma candidature spontanée pour un poste au sein de 
votre entreprise en tant que jeune diplômée en master de Traitement Automatique du Langage (TAL). 
Actuellement à la recherche d’opportunités professionnelles, je suis enthousiaste à l’idée de mettre à 
votre disposition mes connaissances ainsi que mes compétences. 
Au cours de mon parcours académique, j’ai développé plusieurs compétences. Dans un premier 
temps, ma capacité à mener à bien un état de l’art dans le domaine étudié a été démontrée lors de mon 
mémoire de recherche. Cette expérience a permis d’affi

# **Exercice 2 : Utilisation d’un agent avec LangChain**
Dans ce second exercice, vous allez apprendre à utiliser des agents dans LangChain. Les agents permettent d'automatiser des tâches complexes en utilisant des modèles de langage couplés à divers outils. Vous allez configurer un agent pour interroger Wikipedia et obtenir des résumés d'articles.

**Instructions :**
1. Initialiser un agent avec un outil : Vous allez créer un agent capable d'interroger Wikipedia via une API.
2. Configurer le modèle LLM : Comme dans l'exercice précédent, utilisez un modèle GPT pour permettre à l'agent de traiter les demandes.
3. Interroger l'agent : Posez une question à l'agent et récupérez un résumé d'une page Wikipedia sur un sujet spécifique.

In [31]:
!pip install wikipedia


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [32]:
import wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

In [33]:
# 1. Définir une fonction pour interroger Wikipedia
def wiki_search(query: str) -> str:
    try:
        # Recherche de la page Wikipedia la plus pertinente
        page = wikipedia.page(query)
        # Retourner un résumé court (les 3 premiers paragraphes)
        summary = "\n".join(page.content.split('\n')[:3])
        return summary
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Le sujet '{query}' est ambigu, voici quelques options : {e.options[:5]}"
    except wikipedia.exceptions.PageError:
        return f"Le sujet '{query}' n'a pas été trouvé sur Wikipedia."
    except Exception as e:
        return f"Erreur lors de la recherche Wikipedia : {str(e)}"

# 2. Créer un outil LangChain autour de cette fonction
wiki_tool = Tool(
    name="Wikipedia",
    func=wiki_search,
    description="Utilisé pour rechercher des informations sur Wikipedia."
)

# 3. Initialiser le modèle LLM (ici OpenAI)
llm = OpenAI(temperature=0)

# 4. Initialiser l'agent avec l'outil Wikipedia
agent = initialize_agent(
    tools=[wiki_tool],
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# 5. Poser une question à l'agent
question = "Peux-tu me faire un résumé de la Tour Eiffel ?"

# 6. Lancer la requête à l'agent
response = agent.run(question)

print("Réponse de l'agent :\n")
print(response)



> Entering new AgentExecutor chain...
 Je devrais utiliser Wikipedia pour trouver des informations sur la Tour Eiffel.
Action: Wikipedia
Action Input: "Tour Eiffel"
Observation: The Eiffel Tower (  EYE-fəl; French: Tour Eiffel [tuʁ ɛfɛl] ) is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower from 1887 to 1889.
Locally nicknamed "La dame de fer" (French for "Iron Lady"), it was constructed as the centrepiece of the 1889 World's Fair, and to crown the centennial anniversary of the French Revolution. Although initially criticised by some of France's leading artists and intellectuals for its design, it has since become a global cultural icon of France and one of the most recognisable structures in the world. The tower received 5,889,000 visitors in 2022. The Eiffel Tower is the most visited monument with an entrance fee in the world: 6.91 million people ascended it in 2015. It was 